## __Create the machine learning algorithm__

### __Import the relevant libraries__

In [1]:
import numpy as np
import tensorflow as tf

### __Data__

In [2]:
# A temporary variable npz, where we will store each of the three Audiobooks datasets
npz = np.load('Audiobooks_data_train.npz')

# We extract the inputs using the keyword under which we saved them.
train_inputs = npz['inputs'].astype(np.float)  # to ensure that they are all floats
# targets must be int because of sparse_categorical_crossentropy (we want to be able to smoothly one-hot encode them).
train_targets = npz['targets'].astype(np.int)


npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

### __Model__
Outline, optimizers, loss, early stopping and training

In [4]:
input_size = 10
output_size = 2
hidden_layer_size = 50   # Use same hidden layer size for both hidden layers. Not a necessity.
    
# define how the model will look like
model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),  # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),  # 2nd hidden layer
    tf.keras.layers.Dense(output_size, activation='softmax')  # output layer
])


### Choose the optimizer and the loss function
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training
batch_size = 100
max_epochs = 100

# set an early stopping mechanism, let's set patience=2, to be a bit tolerant against random validation loss increases.
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
          train_targets,
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_targets),
          verbose = 2
          )  

Train on 3579 samples, validate on 447 samples
Epoch 1/100
3579/3579 - 0s - loss: 0.6266 - accuracy: 0.6527 - val_loss: 0.5333 - val_accuracy: 0.7271
Epoch 2/100
3579/3579 - 0s - loss: 0.4697 - accuracy: 0.7734 - val_loss: 0.4435 - val_accuracy: 0.7606
Epoch 3/100
3579/3579 - 0s - loss: 0.4141 - accuracy: 0.7837 - val_loss: 0.4113 - val_accuracy: 0.7875
Epoch 4/100
3579/3579 - 0s - loss: 0.3878 - accuracy: 0.8041 - val_loss: 0.3914 - val_accuracy: 0.7987
Epoch 5/100
3579/3579 - 0s - loss: 0.3711 - accuracy: 0.8002 - val_loss: 0.3800 - val_accuracy: 0.8076
Epoch 6/100
3579/3579 - 0s - loss: 0.3620 - accuracy: 0.8125 - val_loss: 0.3789 - val_accuracy: 0.7942
Epoch 7/100
3579/3579 - 0s - loss: 0.3563 - accuracy: 0.8131 - val_loss: 0.3662 - val_accuracy: 0.8188
Epoch 8/100
3579/3579 - 0s - loss: 0.3520 - accuracy: 0.8145 - val_loss: 0.3636 - val_accuracy: 0.8098
Epoch 9/100
3579/3579 - 0s - loss: 0.3457 - accuracy: 0.8153 - val_loss: 0.3601 - val_accuracy: 0.8076
Epoch 10/100
3579/3579 - 0

## __Test the model__

It is very important to realize that fiddling with the hyperparameters overfits the validation dataset. 

The test is the absolute final instance. You should not test before you are completely done with adjusting your model.

If you adjust your model after testing, you will start overfitting the test dataset, which will defeat its purpose.

In [5]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

448/448 [==============================] - 0s 257us/sample - loss: 0.3220 - accuracy: 0.8125


In [6]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.32. Test accuracy: 81.25%
